In [3]:
import numpy as np
import os 

In [4]:
script_dir = os.getcwd()
project_root = os.path.dirname(script_dir)
data_path = os.path.join(project_root, 'data', 'synthetic', 'gaussian_pm_0')

adjacency_path = os.path.join(data_path, 'adjacency_matrix.npy')
A = np.load(adjacency_path)
print(f"Adjacency matrix shape: {A.shape}")

degree_values = np.sum(A, axis=1)
D = np.diag(degree_values)
print(f"Degree matrix shape: {D.shape}")

n = D.shape[0]
P = 0.5 * (np.eye(n) + A @ np.linalg.inv(D))

signal_path = os.path.join(data_path, 'graph_signals.npy')
X = np.load(signal_path)
print(f"Signal matrix shape: {X.shape}")

def compute_c_terms(P, x, num_terms):
    n = P.shape[0]
    I = np.eye(n)
    c_terms = []
    c0 = (I - P) @ x
    c_terms.append(c0)
    P_exp = P.copy()
    for j in range(1, num_terms):
        cj = P_exp @ ((I + P_exp) @ c_terms[-1])
        c_terms.append(cj)
        P_exp_next = P_exp @ P_exp
        P_exp[:] = P_exp_next
    return c_terms


X=X.T
x = X[:, 4]
print(x.shape)
num_terms = 4
c_terms = compute_c_terms(P, x, num_terms)

for idx, c in enumerate(c_terms):
    print(f"c_{idx+1}: {c}")


Adjacency matrix shape: (100, 100)
Degree matrix shape: (100, 100)
Signal matrix shape: (400, 100)
(100,)
c_1: [-4.15617133e-19 -4.04401031e-02 -2.90201426e-04 -2.00512379e-08
 -4.79591887e-16 -3.89061864e-09 -2.52579520e-03 -1.21972349e-06
 -1.17032155e-02 -4.07844412e-19 -2.05095434e-06 -5.28562902e-02
 -5.83826336e-18 -2.24499931e-14 -2.42816102e-11 -1.36781887e-05
 -4.66421502e-14 -4.48273249e-19 -1.48272207e-06 -1.64570993e-12
 -2.13816259e-08 -4.46290976e-14 -9.26557186e-02 -4.43504533e-05
  4.61548262e-02 -6.14524769e-03 -6.94123341e-07 -2.83925886e-14
 -1.78042143e-03 -6.95610380e-02 -2.83909862e-14 -3.79281465e-02
 -1.17082482e-02 -3.78558298e-08 -1.46477467e-06 -2.50233867e-19
 -1.00553201e-14  1.50532064e-01 -5.44682386e-02  2.26018157e-01
 -1.39866576e-03 -8.57894663e-03 -6.52620038e-06 -1.03375055e-02
 -8.68549802e-10 -2.20972763e-09 -1.55978951e-04 -4.69628975e-05
 -5.15237865e-06 -3.57488929e-05  3.22879959e-01 -1.90774852e-08
 -1.93398491e-06 -4.42291308e-04 -8.84549809

In [5]:
import numpy as np

def compute_c_terms(P, x, num_terms):
    n = P.shape[0]
    I = np.eye(n)
    c_terms = []
    c0 = (I - P) @ x
    c_terms.append(c0)
    v_prev = P @ c0
    for j in range(1, num_terms):
        pow = 2 ** (j - 1)
        v_curr = v_prev.copy()
        for _ in range(pow):
            v_curr = P @ v_curr
        cj = v_prev + v_curr
        c_terms.append(cj)
        v_prev = v_curr
    return c_terms

num_terms = 4
c_terms = compute_c_terms(P, x, num_terms)

for idx, c in enumerate(c_terms):
    print(f"c_{idx+1}: {c}")



c_1: [-4.15617133e-19 -4.04401031e-02 -2.90201426e-04 -2.00512379e-08
 -4.79591887e-16 -3.89061864e-09 -2.52579520e-03 -1.21972349e-06
 -1.17032155e-02 -4.07844412e-19 -2.05095434e-06 -5.28562902e-02
 -5.83826336e-18 -2.24499931e-14 -2.42816102e-11 -1.36781887e-05
 -4.66421502e-14 -4.48273249e-19 -1.48272207e-06 -1.64570993e-12
 -2.13816259e-08 -4.46290976e-14 -9.26557186e-02 -4.43504533e-05
  4.61548262e-02 -6.14524769e-03 -6.94123341e-07 -2.83925886e-14
 -1.78042143e-03 -6.95610380e-02 -2.83909862e-14 -3.79281465e-02
 -1.17082482e-02 -3.78558298e-08 -1.46477467e-06 -2.50233867e-19
 -1.00553201e-14  1.50532064e-01 -5.44682386e-02  2.26018157e-01
 -1.39866576e-03 -8.57894663e-03 -6.52620038e-06 -1.03375055e-02
 -8.68549802e-10 -2.20972763e-09 -1.55978951e-04 -4.69628975e-05
 -5.15237865e-06 -3.57488929e-05  3.22879959e-01 -1.90774852e-08
 -1.93398491e-06 -4.42291308e-04 -8.84549809e-10 -6.08203936e-08
 -5.24078194e-08 -1.79873074e-11 -1.02541309e-03 -3.91747130e-02
 -2.21510462e-02 -9.